In [22]:
import sys
import warnings
import metatensor
from itertools import product
from ase.io import read

import numpy as np
from metatensor import (
    Labels,
    TensorBlock,
    TensorMap,
)

from scipy.spatial.transform import Rotation as R
from tqdm.auto import tqdm
from skmatter.preprocessing import StandardFlexibleScaler

from anisoap.representations.radial_basis import (
    GTORadialBasis,
    MonomialBasis,
)
from anisoap.representations.ellipsoidal_density_projection import (
    EllipsoidalDensityProjection,
)

import matplotlib.pyplot as plt

In [23]:
ell_frames = read("ellipsoids.xyz", "0:2")
ell_frames_translation = read("ellipsoids.xyz", "0:2")
ell_frames_rotation = read("ellipsoids.xyz", "0:2")

# This is to make sure the ell_frames list calls c_diameter[] rather than c_diameter and to update the diameters of ellipsoids to be 3,3, and 1.


def update_diameters_and_variablename(frames):
    for frame in frames:
        for i in range(1, 4):
            old = f"c_diameter{i}"
            new = f"c_diameter[{i}]"
            if old in frame.arrays:
                frame.arrays[new] = frame.arrays[old]
            frame.arrays[new] = np.ones(len(frame)) * (3.0 if i < 3 else 1.0)


update_diameters_and_variablename(ell_frames)
update_diameters_and_variablename(ell_frames_translation)
update_diameters_and_variablename(ell_frames_rotation)

In [24]:
# Translation vector is used to demonstrate the power spectrum of ellipsoidal representations are invariant of translation in positions.

translation_vector = np.array([2.0, 2.0, 2.0])
for frame in ell_frames_translation:
    frame.arrays["positions"] += translation_vector

In [25]:
lmax = 5
nmax = 3

AniSOAP_HYPERS = {
    "max_angular": lmax,
    "max_radial": nmax,
    "radial_basis_name": "gto",
    "rotation_type": "quaternion",
    "rotation_key": "c_q",
    "cutoff_radius": 7.0,
    "radial_gaussian_width": 1.5,
    "basis_rcond": 1e-8,
    "basis_tol": 1e-4,
}

In [26]:
calculator = EllipsoidalDensityProjection(**AniSOAP_HYPERS)
power_spectrum_original = calculator.power_spectrum(ell_frames, AniSOAP_HYPERS)
power_spectrum_translated = calculator.power_spectrum(
    ell_frames_translation, AniSOAP_HYPERS
)

# Compare the power_specturm_original and power_spectrum_translated to determine if the power spectrum is translation invariant

if np.allclose(power_spectrum_original, power_spectrum_translated):
    print("Power spectrum has translation invariance")
else:
    print("Power spectrum has no translation invariance")

/Users/PrettyBoy/Desktop/AniSOAP/AniSOAP/anisoap/representations/ellipsoidal_density_projection.py:547: UserWarning: In quaternion mode, quaternions are assumed to be in (w,x,y,z) format.
  warnings.warn(


Computing neighborlist:   0%|          | 0/2 [00:00<?, ?it/s]

Iterating samples for (0, 0):   0%|          | 0/33 [00:00<?, ?it/s]

Accruing lmax:   0%|          | 0/6 [00:00<?, ?it/s]

Iterating tensor block keys:   0%|          | 0/6 [00:00<?, ?it/s]

...

Contracting features:   0%|          | 0/1 [00:00<?, ?it/s]

Iterating neighbor types:   0%|          | 0/1 [00:00<?, ?it/s]

Finding matching block samples:   0%|          | 0/4 [00:00<?, ?it/s]

Contracting features:   0%|          | 0/1 [00:00<?, ?it/s]

Power spectrum has translation invariance


In [27]:
ell_frames_rotation[0].arrays["c_q"]

array([[ 0.15965019,  0.67170996, -0.07507814,  0.71950039],
       [-0.213207  , -0.03290243,  0.26500539,  0.93980442]])

In [28]:
quaternion = [1, 2, 0, -3]

q_rotation = R.from_quat(quaternion, scalar_first=True)

for frames in ell_frames_rotation:
    frames.arrays["c_q"] = R.as_quat(
        q_rotation * R.from_quat(frames.arrays["c_q"], scalar_first=True),
        scalar_first=True,
    )

In [29]:
power_spectrum_rotation = calculator.power_spectrum(ell_frames_rotation, AniSOAP_HYPERS)

# Compare the power_specturm_original and power_spectrum_rotation to determine if the power spectrum is rotation invariant
if np.allclose(power_spectrum_original, power_spectrum_rotation):
    print("Power spectrum has rotation invariance")
else:
    print("Power spectrum has no rotation invariance")

/Users/PrettyBoy/Desktop/AniSOAP/AniSOAP/anisoap/representations/ellipsoidal_density_projection.py:547: UserWarning: In quaternion mode, quaternions are assumed to be in (w,x,y,z) format.
  warnings.warn(


Computing neighborlist:   0%|          | 0/2 [00:00<?, ?it/s]

Iterating samples for (0, 0):   0%|          | 0/33 [00:00<?, ?it/s]

Accruing lmax:   0%|          | 0/6 [00:00<?, ?it/s]

Iterating tensor block keys:   0%|          | 0/6 [00:00<?, ?it/s]

...

Contracting features:   0%|          | 0/1 [00:00<?, ?it/s]

Iterating neighbor types:   0%|          | 0/1 [00:00<?, ?it/s]

Finding matching block samples:   0%|          | 0/4 [00:00<?, ?it/s]

Contracting features:   0%|          | 0/1 [00:00<?, ?it/s]

Power spectrum has no rotation invariance
